In [3]:
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;                              
open Microsoft.Quantum.Math;

In [4]:
operation ReflectAboutAllOnes(register : Qubit[]) : Unit is Adj + Ctl {
    Controlled Z(Most(register), Tail(register));
}

ReflectAboutAllOnes

In [5]:
operation PrepareAllOnes(register : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(X, register);
}

PrepareAllOnes

In [6]:
operation PrepareInitialState(register : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(H, register);
}
 
operation ReflectAboutInitialState(
    prepareInitialState : (Qubit[] => Unit is Adj),
    register : Qubit[]
)
: Unit {
    within {
        Adjoint prepareInitialState(register);
        PrepareAllOnes(register);
    } apply {
        ReflectAboutAllOnes(register);
    }
}

PrepareInitialState, ReflectAboutInitialState

In [7]:
operation ReflectAboutMarkedState(
    markedItemOracle : ((Qubit[], Qubit) => Unit is Adj),
    inputQubits : Qubit[])
: Unit is Adj {
    using (flag = Qubit()) {
        within {
            X(flag);
            H(flag);
        } apply{
            markedItemOracle(inputQubits, flag);
        }
    }
}

ReflectAboutMarkedState

In [8]:
function NIterations(nQubits : Int) : Int {
    let nItems = 1 <<< nQubits;
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let nIterations = Round(0.25 * PI() / angle - 0.5);
    return nIterations;
}

NIterations

In [10]:
operation ApplyOracle(
    idxMarkedItem : Int,
    register : Qubit[],
    flag : Qubit)
: Unit is Adj + Ctl {
    (ControlledOnInt(idxMarkedItem, X))(register, flag);
}

ApplyOracle

In [11]:
operation SearchForMarkedItem(
    nQubits : Int,
    markItem : ((Qubit[], Qubit) => Unit is Adj)
)
: Int {
    using (qubits = Qubit[nQubits]) {
        PrepareInitialState(qubits);
 
        for (idxIteration in 0..NIterations(nQubits) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(PrepareInitialState, qubits);
        }
 
        return MeasureInteger(LittleEndian(qubits));
    }
}

SearchForMarkedItem

In [12]:
operation RunGroverSearch() : Unit {
    let idxMarkedItem = 6;
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItem(3, markItem);
    Message($"marked {idxMarkedItem} and found {foundItem}.");
}

RunGroverSearch

In [15]:
%simulate RunGroverSearch

marked 6 and found 6.


()